In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

#Select between CH9 and CH11
path = "angular_distr_el/CH9/"
# path = "angular_distr_el/CH11/"

#move first the ID you need
IDs = ["ID1_mol_e0_valid/EN_gate/","ID_ALL_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
dirs = ["MFPADs_multinew","MFPADs_multinew_phi"]

loc=path+IDs[0]+dirs[0]

##550eV - 11.5eV CH9
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CR_Knov.root")
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH9_550eV_CL_Knov.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CL_Knov.root")

##550eV - 11.5eV CH11
#update nov Kilian binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CR_Knov.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_CH11_550eV_CL_Knov.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CL_Knov.root")

#low new binning (24,12)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

# #high neew binning (36,18)
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")

#546eV - 8.2eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546.5eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546.5eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#544eV - 6.1eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#542eV - 4eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#541eV 3eV
#low new binning
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541eV_CR_9600-3700ns_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541eV_CL_9600-3700ns_multiCH9_MFPAD_30_t0.root")

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
d1=type(check)
check.all_members; #all members of the single histogram

In [ ]:
#Loading of the MFPADS and cos len = 27
MFPAD_RCR,ctheta_RCR,cosphi_photon,MFPAD_axis,ctheta_axis,ctheta_axis_red=fugi.import_MFPAD(fileRCR, loc, full=True)
MFPAD_RCL,ctheta_RCL=fugi.import_MFPAD(fileRCL, loc)
#projection of cos on x axis: len = 18
cosx_RCR=fugi.projection(MFPAD_RCR,a=0)
cosx_RCL=fugi.projection(MFPAD_RCL,a=0)

if fileSCL and fileSCR is not None:    
    MFPAD_SCR,ctheta_SCR=fugi.import_MFPAD(fileSCR, loc)
    MFPAD_SCL,ctheta_SCL=fugi.import_MFPAD(fileSCL, loc)
    #projection of cos on x axis
    cosx_SCR=fugi.projection(MFPAD_SCR,a=0)
    cosx_SCL=fugi.projection(MFPAD_SCL,a=0)

In [ ]:
#COORDIANTES PHOTON ROTATION
#adjusting cosphi with a shifting to have it equal to theory
#before sorting it is natuarally cos sorted from experiment
cosphi_adj=[]
ctheta_axis_red=ctheta_axis_red[0];

phiM=(MFPAD_axis[0][0][1:] + MFPAD_axis[0][0][:-1])/2
cosM=(MFPAD_axis[0][1][1:] + MFPAD_axis[0][1][:-1])/2

print("MFPAD_xy shape = ", MFPAD_axis[0][0].shape)
print("phiM shape = ", phiM.shape)
print("cosM shape = ", cosM.shape)

print("cosphi diagnostic: type=",type(cosphi_photon), " shape= ", cosphi_photon.shape)

cos_adj=fugi.shift_func(cosphi_photon[:,0]).tolist();
phi_adj=fugi.shift_func(cosphi_photon[:,1]).tolist();

for cos,phi in zip(cos_adj,phi_adj):
    cosphi_adj.append((cos,phi))
cosphi_adj=np.around(np.array(cosphi_adj),3)

print("cosphi_adj: type=",type(cosphi_adj), " shape= ", cosphi_adj.shape)

print(cosphi_adj[0][0])
#look to the values in b1 map and found the MFPAD

dfind=pd.DataFrame(cosphi_adj, columns=["ctheta","phi"])

#rearranging the photon vector
cosphi_adj_cos=np.array((dfind.sort_values(by=["ctheta","phi"]))[["ctheta","phi"]].values)
cosphi_adj_phi=np.array((dfind.sort_values(by=["phi","ctheta"]))[["ctheta","phi"]].values)

xgo_phi,ygo_phi = fugi.create_gocoords(a=0,reduced=True)
xgo_cos,ygo_cos = fugi.create_gocoords(a=1,reduced=True)

print("xgo_phi length = ", len(xgo_phi))
print("xgo_cos length = ", len(xgo_cos))

print("cosphi_adj[3] = ", cosphi_adj[3][0],cosphi_adj[3][1])
print("cosphi_adj_cos[3] = ", cosphi_adj_cos[3][0],cosphi_adj_cos[3][1])
print("cosphi_adj_phi[3] = ", cosphi_adj_phi[3][0],cosphi_adj_phi[3][1])

In [ ]:
MFPAD_RCR.shape

In [ ]:
MFPAD_RCR_cos=fugi.sorting_array(MFPAD_RCR, cosphi_adj, a=1)
MFPAD_RCL_cos=fugi.sorting_array(MFPAD_RCL, cosphi_adj, a=1)
MFPAD_RCR_phi=fugi.sorting_array(MFPAD_RCR, cosphi_adj, a=2)
MFPAD_RCL_phi=fugi.sorting_array(MFPAD_RCL, cosphi_adj, a=2)

if fileSCL and fileSCR is not None:
    MFPAD_SCR_cos=fugi.sorting_array(MFPAD_SCR, cosphi_adj, a=1)
    MFPAD_SCL_cos=fugi.sorting_array(MFPAD_SCL, cosphi_adj, a=1)
    MFPAD_SCR_phi=fugi.sorting_array(MFPAD_SCR, cosphi_adj, a=2)
    MFPAD_SCL_phi=fugi.sorting_array(MFPAD_SCL, cosphi_adj, a=2)

In [ ]:
print(MFPAD_RCR.shape)
print(MFPAD_RCR.max(),MFPAD_RCR.min())

In [ ]:
#flipping; axis=1 flips along cos(theta), axis=0 along phi
#there is a smart (and correct) flipping reccomended by Kilian, the following method is WRONG
MFPAD_RCL_flip=[]
MFPAD_RCR_flip=[]

for el in MFPAD_RCL:
    # MFPAD_RCL_flip.append(np.flip(np.flip(el, axis=0),axis=1))
    MFPAD_RCL_flip.append(np.flip(el, axis=1))
MFPAD_RCL_flip=np.array(MFPAD_RCL_flip)
MFPAD_RCL_flip_norm=fugi.normalise_matrix(MFPAD_RCL_flip,normtype=1)

for el in MFPAD_RCR:
    # MFPAD_RCR_flip.append(np.flip(np.flip(el, axis=0),axis=1))
    MFPAD_RCR_flip.append(np.flip(el, axis=1))
MFPAD_RCR_flip=np.array(MFPAD_RCR_flip)
MFPAD_RCR_flip_norm=fugi.normalise_matrix(MFPAD_RCR_flip,normtype=1)

#best to do that at the end?
MFPAD_RCL_flip_cos=fugi.sorting_array(MFPAD_RCL_flip, cosphi_adj, a=1)
MFPAD_RCL_flip_phi=fugi.sorting_array(MFPAD_RCL_flip, cosphi_adj, a=2)
MFPAD_RCR_flip_cos=fugi.sorting_array(MFPAD_RCR_flip, cosphi_adj, a=1)
MFPAD_RCR_flip_phi=fugi.sorting_array(MFPAD_RCR_flip, cosphi_adj, a=2)


if fileSCL and fileSCR is not None:
    MFPAD_SCL_flip=[]
    MFPAD_SCR_flip=[]

    for el in MFPAD_SCL:
        # MFPAD_SCL_flip.append(np.flip(np.flip(el, axis=0),axis=1))
        MFPAD_SCL_flip.append(np.flip(el, axis=1))
    MFPAD_SCL_flip=np.array(MFPAD_SCL_flip)
    MFPAD_SCL_flip_norm=fugi.normalise_matrix(MFPAD_SCL_flip,normtype=1)

    for el in MFPAD_SCR:
        # MFPAD_SCR_flip.append(np.flip(np.flip(el, axis=0),axis=1))
        MFPAD_SCR_flip.append(np.flip(el, axis=1))
    MFPAD_SCR_flip=np.array(MFPAD_SCR_flip)
    MFPAD_SCR_flip_norm=fugi.normalise_matrix(MFPAD_SCR_flip,normtype=1)

    #best to do that at the end?
    MFPAD_SCL_flip_cos=fugi.sorting_array(MFPAD_SCL_flip, cosphi_adj, a=1)
    MFPAD_SCL_flip_phi=fugi.sorting_array(MFPAD_SCL_flip, cosphi_adj, a=2)
    MFPAD_SCR_flip_cos=fugi.sorting_array(MFPAD_SCR_flip, cosphi_adj, a=1)
    MFPAD_SCR_flip_phi=fugi.sorting_array(MFPAD_SCR_flip, cosphi_adj, a=2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
ax1.contourf(phiM,cosM,MFPAD_RCR[0].T)
ax2.contourf(phiM,cosM,MFPAD_RCR_flip[0].T)

# MFPAD_SCL_flip_cos[45]==np.flip(np.flip(MFPAD_SCL_cos[45],axis=1),axis=0) #True

In [ ]:
#sum within an experimemt axis=0 is among the 72 elements
#single enantiomer and helicity
MFPAD_RCR_sum=np.sum(MFPAD_RCR, axis=0)
MFPAD_RCL_sum=np.sum(MFPAD_RCL, axis=0)

#normalised
MFPAD_RCR_norm=fugi.normalise_matrix(MFPAD_RCR,normtype=1)
MFPAD_RCL_norm=fugi.normalise_matrix(MFPAD_RCL,normtype=1)
MFPAD_RCR_norm_sum=np.sum(MFPAD_RCR_norm,axis=0)
MFPAD_RCL_norm_sum=np.sum(MFPAD_RCL_norm,axis=0)

if fileSCL and fileSCR is not None:
    #single enantiomer and helicity
    MFPAD_SCR_sum=np.sum(MFPAD_SCR, axis=0)
    MFPAD_SCL_sum=np.sum(MFPAD_SCL, axis=0)
    MFPAD_SCR_flip_sum=np.sum(MFPAD_SCR_flip, axis=0)
    MFPAD_SCL_flip_sum=np.sum(MFPAD_SCL_flip, axis=0)

    #normalised
    MFPAD_SCR_norm=fugi.normalise_matrix(MFPAD_SCR,normtype=1)
    MFPAD_SCL_norm=fugi.normalise_matrix(MFPAD_SCL,normtype=1)
    MFPAD_SCR_norm_sum=np.sum(MFPAD_SCR_norm,axis=0)
    MFPAD_SCL_norm_sum=np.sum(MFPAD_SCL_norm,axis=0)
    MFPAD_SCR_flip_norm_sum=np.sum(MFPAD_SCR_flip_norm, axis=0)
    MFPAD_SCL_flip_norm_sum=np.sum(MFPAD_SCL_flip_norm, axis=0)

    #sum of two enantiomers: there is no influence of the ellicity
    MFPAD_tot_sum=MFPAD_RCR_sum+MFPAD_RCL_sum+MFPAD_SCR_flip_sum+MFPAD_SCL_flip_sum
    # MFPAD_tot_sum=MFPAD_RCR_sum+MFPAD_RCL_sum+MFPAD_SCR_sum+MFPAD_SCL_sum
    #normalised
    MFPAD_tot_norm_sum=MFPAD_RCR_norm_sum+MFPAD_RCL_norm_sum+MFPAD_SCR_flip_norm_sum+MFPAD_SCL_flip_norm_sum
    # MFPAD_tot_norm_sum=MFPAD_RCR_norm_sum+MFPAD_RCL_norm_sum+MFPAD_SCR_norm_sum+MFPAD_SCL_norm_sum

#test samrt sum
counts=0
for j in range(12):
    for i in range(6):
        if j>5:  
            print("b=",i,j,"a1=",5-i,j-6)
            i+=1
        else:
            print("b=",i,j,"a2=",5-i,j+6)
            i+=1
        counts+=1
print(counts)
#check the reshape(6,12,-1)
cosphi_adj_cos.reshape(6,12,-1)[:13]


In [ ]:
#smart flipping for the single 72 MFPAD image. Has to be in cos, do the phi just as a check
MFPAD_RCLCR=[]

#sum helicities: -cos(theta), phi+180 in photn coordiantes
for j in range(12):
    for i in range(6):
        if j>5:
            MFPAD_RCLCR.append(MFPAD_RCL_phi.reshape(6,12,-1)[i,j]+MFPAD_RCR_phi.reshape(6,12,-1)[5-i,j-6])
            i+=1
        else:
            MFPAD_RCLCR.append(MFPAD_RCL_phi.reshape(6,12,-1)[i,j]+MFPAD_RCR_phi.reshape(6,12,-1)[5-i,6+j])
            i+=1
MFPAD_RCLCR=np.array(MFPAD_RCLCR_phi).reshape(72,MFPAD_RCL_phi.shape[1],MFPAD_RCL_phi.shape[2])
MFPAD_RCLCR_cos=fugi.sorting_array(MFPAD_RCLCR, cosphi_adj, a=1)
MFPAD_RCLCR_phi=fugi.sorting_array(MFPAD_RCLCR, cosphi_adj, a=2) #equal to the original!


if fileSCL and fileSCR is not None:
    MFPAD_SCLCR=[]
    MFPAD_totordered=[]
    #sum helicities: -cos(theta), phi+180 in photn coordiantes
    for j in range(12):
        for i in range(6):
            if j>5:
                MFPAD_SCLCR.append(MFPAD_SCL_phi.reshape(6,12,-1)[i,j]+MFPAD_SCR_phi.reshape(6,12,-1)[5-i,j-6])
                i+=1
            else:
                MFPAD_SCLCR.append(MFPAD_SCL_phi.reshape(6,12,-1)[i,j]+MFPAD_SCR_phi.reshape(6,12,-1)[5-i,6+j])
                i+=1
    MFPAD_SCLCR=np.array(MFPAD_SCLCR).reshape(72,MFPAD_SCL_phi.shape[1],MFPAD_SCL_phi.shape[2])
    MFPAD_SCLCR_cos=fugi.sorting_array(MFPAD_SCLCR, cosphi_adj, a= 1)
    MFPAD_SCLCR_phi=fugi.sorting_array(MFPAD_SCLCR, cosphi_adj, a= 2) #equal to the original!
    
    #sum enantiomer: -cos(theta) fliped axis=1 cos!, -cos(theta) photon coordinates
    nphi=MFPAD_SCL.shape[1];ncos=MFPAD_SCL.shape[2]
    for j in range(12):
        for i in range(6):
            MFPAD_totordered.append(MFPAD_SCLCR_phi.reshape(6,12,nphi,ncos)[i,j]+np.flip(MFPAD_RCLCR_phi.reshape(6,12,nphi,ncos)[5-i,j], axis=1))
            i+=1

    MFPAD_totordered=np.array(MFPAD_totordered).reshape(72,MFPAD_SCL.shape[1],MFPAD_SCL.shape[2])
    MFPAD_totordered_cos=fugi.sorting_array(MFPAD_totordered, cosphi_adj, a=1)
    MFPAD_totordered_phi=fugi.sorting_array(MFPAD_totordered, cosphi_adj, a=2) #equal to the original!

In [ ]:
#SUM of S enantiomers two different helicities
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    #old adding up
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCLCR_cos[i],0.3,0.6).T, levels=5)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_SCLCR_cos_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#SUM of S enantiomers two different helicities
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    #old adding up
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCLCR_cos[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_RCLCR_cos_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#TOTAL sum of enantiomers and helicities
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_totordered_cos[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi_adj[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_total_cos.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#unsorted non-smoothed version

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCR_cos[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_SCR_cos_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
#unsorted non-smoothed version

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCL_cos[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi_adj[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_SCL_cos_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCR_phi[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_SCR_phi_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCL_phi[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
fig.savefig('../PYTHON_graphs/OUTPUTS/MFPAD_CH9_SCL_phi_exp.png', figsize=(50, 24), dpi=144, transparent=False)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.imshow(fugi.smoothgauss(MFPAD_RCR_cos[i],0.3,0.6).T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap="viridis", aspect="auto")
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_phi[i].T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)

fugi.overlaygraph(fig)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_RCR_phi[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    # ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fugi.overlaygraph(fig)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCR_phi[i],0.3,0.6).T)
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fugi.overlaygraph(fig)
plt.show()

In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,fugi.smoothgauss(MFPAD_SCL_phi[i],0.3,0.6).T)    
    ax.tick_params(labelsize=12)
    #ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fugi.overlaygraph(fig)
plt.show()

#a try with polotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

endrange=[]

for MFPAD in MFPAD_RCR_cos:
    endrange.append(np.amax(fugi.smoothgauss(MFPAD,0.3,0.6)))
endrange=np.amax(endrange)


fig = go.Figure()
fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=fugi.smoothgauss(MFPAD_RCR_cos[j+i*6],0.3,0.6).T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=endrange, size=endrange/10)), row=j+1, col=i+1)
    
fig.update_layout(
    title=dict(
        text="MFPAD smoothed cos sorted" ,
        #y=0.9,
        x=0.5,
        xanchor="center",
        yanchor="top"),
    #coloraxis=dict(colorscale=Seismic_r),
    #showlegend=False,
    autosize=True,
    width=1000,
    #height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
fig,ax = plt.subplots(2,4, figsize=(20, 10))
ax[0][0].contourf(phiM,cosM,MFPAD_RCR_sum.T)
ax[0][0].set_title("MFPAD RCR")
ax[0][1].contourf(phiM,cosM,MFPAD_SCR_flip_sum.T)
ax[0][1].set_title("MFPAD SCR_flip")
ax[0][2].contourf(phiM,cosM,MFPAD_RCL_sum.T)
ax[0][2].set_title("MFPAD RCL")
ax[0][3].contourf(phiM,cosM,MFPAD_SCL_flip_sum.T)
ax[0][3].set_title("MFPAD SCL_flip")

ax[1][0].contourf(phiM,cosM,MFPAD_RCR_norm_sum.T)
ax[1][0].set_title("norm")
ax[1][1].contourf(phiM,cosM,MFPAD_SCR_flip_norm_sum.T)
ax[1][1].set_title("norm")
ax[1][2].contourf(phiM,cosM,MFPAD_RCL_norm_sum.T)
ax[1][2].set_title("norm")
ax[1][3].contourf(phiM,cosM,MFPAD_SCL_flip_norm_sum.T, )
ax[1][3].set_title("norm")
plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(20, 10))
ax1.contourf(phiM,cosM,MFPAD_SCR_flip_sum.T, levels=25)
ax1.set_title("MFPAD tot enant")

ax2.contourf(phiM,cosM,MFPAD_SCR_flip_norm_sum.T,levels=25)
ax2.set_title("norm")
plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2, figsize=(20, 10))
cs=ax1.contourf(phiM,cosM,MFPAD_tot_sum.T, levels=15)
ax1.set_title("MFPAD tot enant")

cs1=ax2.contourf(phiM,cosM,MFPAD_tot_norm_sum.T,levels=15)
ax2.set_title("norm")
fig.colorbar(cs)
plt.show()

In [ ]:
#a try to combine the flipping an the smart sum

fig,ax = plt.subplots(2,2, figsize=(11, 11))

ax[0][0].imshow(MFPAD_SR_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
ax[0][0].set_title("MFPAD SR enant")
ax[1][0].imshow(MFPAD_SR_norm_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
ax[1][0].set_title("norm")

ax[0][1].imshow(MFPAD_SCLCR_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
ax[0][1].set_title("MFPAD sum")
ax[1][1].imshow(MFPAD_SCLCR_norm_sum.T, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='quadric', cmap=cmap_temp, aspect="auto")
ax[1][1].set_title("norm")


plt.show()

In [ ]:
#cos(theta) seems inverted rispect to contour
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("MFPAD SR norm", "MFPAD sum norm"))

# fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_sum_SR_norm_sum.T, line_smoothing=0.85, coloraxis="coloraxis1"), 1, 1)
fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_sum_SCLCR_norm_sum.T, line_smoothing=0.85, coloraxis="coloraxis2"), 1, 2)

fig.update_layout(
    coloraxis1=dict(colorscale='Viridis',
                    colorbar=dict(x=0.515, xanchor='right', thickness=10)),
    coloraxis2=dict(colorscale='Viridis',
                    colorbar=dict(x=1.0, xanchor='left', thickness=10)),
    # showlegend=False,
    autosize=False,
    width=1000,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()